In [26]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
from mediapipe.framework.formats import landmark_pb2

In [8]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# 1) 사용자의 입술 크기 측정

In [18]:
land_list = [61, 0, 291, 17]
text = "Calculating the average width of your mouth..."

In [19]:
time_c = 0.0
sum_w = 0

cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            point_list = []
            for i in land_list:
                point_list.append(results.multi_face_landmarks[0].landmark[i])
            
            a_x = (point_list[0].x - point_list[2].x) ** 2 * 1000
            a_y = (point_list[0].y - point_list[2].y) ** 2 * 1000
            a = a_x + a_y
            
            b_x = (point_list[1].x - point_list[3].x) ** 2 * 1000
            b_y = (point_list[1].y - point_list[3].y) ** 2 * 1000
            b = b_x + b_y
            
            sum_w += a * b * math.pi
            time_c += 1
            
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=landmark_pb2.NormalizedLandmarkList(landmark = point_list),
                connection_drawing_spec=drawing_spec)
            
            image = cv2.flip(image, 1)
            cv2.putText(image, text, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (98, 17, 0), 2, cv2.LINE_AA)
            cv2.imshow(text, image)
            cv2.waitKey(1)
            
            if time_c == 100:
                break
                
cv2.destroyAllWindows()
cap.release()

In [20]:
width_m = sum_w // 100
print("사용자 입술의 크기:", width_m)

사용자 입술의 크기: 53.0


# 2) 하품한 횟수 카운트

In [25]:
cnt_y = 0
mode = "Normal Condition (cnt_y : " + str(cnt_y) + ")"
flag = False
time_s = time.time()

cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            point_list = []
            for i in land_list:
                point_list.append(results.multi_face_landmarks[0].landmark[i])
            
            a_x = (point_list[0].x - point_list[2].x) ** 2 * 1000
            a_y = (point_list[0].y - point_list[2].y) ** 2 * 1000
            a = a_x + a_y
            
            b_x = (point_list[1].x - point_list[3].x) ** 2 * 1000
            b_y = (point_list[1].y - point_list[3].y) ** 2 * 1000
            b = b_x + b_y
            
            width = a * b * math.pi
            ratio = width  / width_m
            
            if ratio > 6 and time.time() - time_s >= 2:
                flag = True
                mode = "Yawning... (ratio : " + str(int(ratio)) + ")"
            else:
                if flag:
                    flag = False
                    time_s = time.time()
                    cnt_y += 1
                    mode = "Normal Condition (cnt_y : " + str(cnt_y) + ")"
            
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=landmark_pb2.NormalizedLandmarkList(landmark = point_list),
                connection_drawing_spec=drawing_spec)
            
            image = cv2.flip(image, 1)
            
            
            cv2.putText(image, mode, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (98, 17, 0), 2, cv2.LINE_AA)
            
            cv2.imshow("Count the number of Yawns", image)
            cv2.waitKey(1)
            
            if cv2.waitKey(1) == 27:
                break
                
cv2.destroyAllWindows()
cap.release()